# SME0110 - Programação Matemática, 2023/2

## Membros

- Giovanni Shibaki Camargo - 11796444
- João Antônio Misson Milhorim - 11834331
- Pedro Kenzo Muramatsu Carmo - 11796451
- Rafael Corona - 4769989
- Vitor Caetano Brustolin - 11795589

## Trabalho de Otimização Inteira - Toy Problem

## Descrição do problema

O problema de desenvolvido visa disponibilizar guarda-chuvas para os alunos do Campus, de maneira a suprir a demanda dos alunos tal que eles andem, em média, o mínimo possível, de forma que o custo do projeto esteja dentro de um orçamento limitado.

$ Min Z = ∑^{n}_{i=1} ∑^{m}_{j=1} dist_{ij} x_{ij} d_j $

s.a
<table class="tg">
<thead>
  <tr>
    <th class="tg-0lax"> $ ∑^{n}_{i=1} x_{ij} = 1 $ </th>
    <th class="tg-0lax">$ j = 1, … , m $</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax">$ ∑^{m}_{j=1} d_j x_{ij} <= cap_p yp_i + cap_g yg_i $</td>
    <td class="tg-0lax">$ i = 1, …, n $</td>
  </tr>
    <tr>
    <td class="tg-0lax">$ yp_i + yg_i <= 1 $</td>
    <td class="tg-0lax">$ i = 1, …, n $</td>
  </tr>
    </tr>
    <tr>
    <td class="tg-0lax">$ ∑^{n}_{i=1} f_g yg_i + f_p yp_i <= c_{max} yg_i $</td>
    <td class="tg-0lax">$ $</td>
  </tr>
    <tr>
    <td class="tg-0lax">$ Cap_i ∈ \{0, 1\} $</td>
    <td class="tg-0lax">$  i = 1, …, n $</td>
  </tr>
  <tr>
    <td class="tg-0lax">$ yp_i ∈ \{0, 1\} $</td>
    <td class="tg-0lax">$ i = 1, …, n $</td>
  </tr>
    <tr>
    <td class="tg-0lax">$ yg_i ∈ \{0, 1\} $</td>
    <td class="tg-0lax">$  i = 1, …, n $</td>
  </tr>
  <tr>
    <td class="tg-0lax">$ 0 <= x_{ij} <= 1 $</td>
    <td class="tg-0lax">$ i = 1, …, n; j = 1, …, m $</td>
  </tr>
</tbody>
</table>

em que:

Dados do problema:

*  $n$: é o número de locais onde podem ser instaladas as facilidades;

*  $m$: é o número de blocos de salas de aula que podem ser atendidos;

*  $f_p$: é o custo fixo da para abrir uma facilidade pequena;  
*  $f_g$: é o custo fixo da para abrir uma facilidade grande;  
*  $cap_p$: é a capacidade de uma facilidade pequena;  
*  $cap_g$: é a capacidade de uma facilidade grande;  

*  $dist_{ij}$ : é a distância do ponto de guarda chuvas i ($i ∈ \{1, ...n\}$) até o bloco j ($j ∈ \{1, ...m\}$);

*  $d_j$: é a demanda mínima por guarda chuvas do bloco j ($j ∈ \{1, ... m\}$);

*  $Cap_i$: Indica se o bloco suporta apenas uma facilidade pequena ou uma pequena ou grande. 0 representa que o ponto suporta apenas uma facilidade pequena, e o valor 1 indica que o ponto comporta uma facilidade pequena ou uma grande ($i ∈ \{1, ...n\}$);

Variáveis do problema:  

*  $yp_i$: é uma variável binária que assume o valor 1 se um ponto pequeno i ($i ∈ \{1, ...n\}$) é aberto, e assume o valor 0 caso contrário;

*  $yg_i$: é uma variável binária que assume o valor 1 se um ponto grande i ($i ∈ \{1, ...n\}$) é aberto, e assume o valor 0 caso contrário;

*  $x_{ij}$ : é uma variável contínua que corresponde a fração de alunos do bloco j ($j ∈ \{1, ... m\}$) atendida pela facilidade i ($i ∈ \{1, ... n\}$) ($0 ≤ xij ≤ 1$).

## Imports necessários

In [64]:
import random
import numpy as np
from itertools import product
import pyscipopt as scip

In [65]:
import random
import numpy as np
from itertools import product

%pip install gurobipy
import gurobipy as gp
from gurobipy import GRB

%pip install gurobipy_pandas
%pip install gurobi-machinelearning
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr

Note: you may need to restart the kernel to use updated packages.Defaulting to user installation because normal site-packages is not writeable




[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Gerando os dados do problema

Lendo o arquivo de entrada contendo os dados necessários para a resolução do Toy Problem, segundo a modelagem acima.

In [66]:
def le_instancia(file):

  with open(file, 'r') as file:
    # Lendo a primeira linha
    primeira_linha = file.readline()
    # Separando os valores de 'n' e 'm' usando espaço como delimitador
    n, m = map(int, primeira_linha.split())

    # Separando os valores de 'f_p' e 'f_g' usando espaço como delimitador
    linha = file.readline()
    f_p, f_g = map(float, linha.split())

    # Separando os valores de 'cap_p' e 'cap_g' usando espaço como delimitador
    linha = file.readline()
    cap_p, cap_g = map(int, linha.split())

    linha = file.readline()
    c_max = float(linha)

    cap_i = [] #capacidades das facilidades
    pares_d_c = [] #demanda de cada cliente m seguido dos custos de transporte da facilidade i para cada cliente j

    # Leia as próximas 'n' linhas, contendo 1 se é possível construir uma facilidade grande, 0 se apenas uma pequena
    for i in range(n):
      linha = file.readline()
      cap_i.append(int(linha))

    # Leia as próximas 'm' linhas, contendo as demandas de cada cliente e custo de transporte
    for i in range(m):
        linha = file.readline().strip()
        valores = list(map(float, linha.split()))

        d_j = valores[0]

        c_i_j = valores[1:(n+1)]

        pares_d_c.append({'d': d_j, 'c': c_i_j})

  d_j = []
  for par in pares_d_c:
    d_j.append(int(par["d"]))

  # Distância que o aluno j deve andar até a facilidade j
  produto_cartesiano = list(product(range(n), range(m)))
  distancia_percorrida = {}

  counter_cliente = 0
  for par in pares_d_c:
    counter_facilidade = 0

    for value in par["c"]:
      key_tuple = (counter_facilidade, counter_cliente)
      distancia_percorrida[key_tuple] = value
      counter_facilidade = counter_facilidade + 1

    counter_cliente = counter_cliente + 1

  return n, m, f_p, f_g, cap_p, cap_g, cap_i, c_max, d_j, distancia_percorrida, produto_cartesiano

In [67]:
n, m, f_p, f_g, cap_p, cap_g, cap_i, c_max, d_j, distancia_percorrida, produto_cartesiano = le_instancia('toy_problem_input_new(1).txt') #lê a instancia do Toy Problem

In [68]:
#testando a leitura
print(n)
print(m)

10
21


In [69]:
print(d_j)

[164, 2, 10, 13, 11, 3, 24, 16, 27, 9, 19, 17, 24, 11, 10, 7, 9, 18, 5, 11, 13]


In [70]:
c_max = 30000

In [71]:
print(d_j)

[164, 2, 10, 13, 11, 3, 24, 16, 27, 9, 19, 17, 24, 11, 10, 7, 9, 18, 5, 11, 13]


In [72]:
print(distancia_percorrida)

{(0, 0): 0.0, (1, 0): 223.60679774997897, (2, 0): 156.20499351813308, (3, 0): 404.4749683231337, (4, 0): 342.34485537247383, (5, 0): 379.47331922020555, (6, 0): 107.70329614269008, (7, 0): 356.08987629529713, (8, 0): 223.60679774997897, (9, 0): 241.66091947189145, (0, 1): 170.88007490635061, (1, 1): 56.568542494923804, (2, 1): 266.8332812825267, (3, 1): 509.9019513592785, (4, 1): 304.63092423455635, (5, 1): 504.7771785649585, (6, 1): 63.24555320336759, (7, 1): 509.9019513592785, (8, 1): 393.95431207184413, (9, 1): 382.099463490856, (0, 2): 223.60679774997897, (1, 2): 0.0, (2, 2): 300.66592756745814, (3, 2): 563.5601121442148, (4, 2): 288.44410203711914, (5, 2): 560.357029044876, (6, 2): 116.61903789690601, (7, 2): 566.3920903402518, (8, 2): 447.21359549995793, (9, 2): 438.6342439892262, (0, 3): 252.98221281347037, (1, 3): 44.721359549995796, (2, 3): 342.34485537247383, (3, 3): 566.0388679233962, (4, 3): 328.02438933713455, (5, 3): 568.5068161420758, (6, 3): 145.60219778561037, (7, 3): 

In [73]:
print(sum(d_j))
print(cap_g*n)

423
600


## Usando PySCIPOpt para a resolução do Toy Problem

In [74]:
def Cria_modelo_minimizacao(n, m, f_p, f_g, cap_p, cap_g, cap_i, c_max, d_j, distancia_percorrida, time_limit):
  model = scip.Model("Localização de facilidades - Minimizando custos")
  model.redirectOutput() #para que o colab consiga printar os resultados

  abre_facilidadeP = {} #indica quais facilidades pequenas serão abertas
  abre_facilidadeG = {} #indica quais facilidades grandes serão abertas

  designa = {} #indica quanto cada facilidade envia a cada cliente
  for f in range(n):
    abre_facilidadeP[f] = model.addVar(vtype="B", name="abreP[%d]"%f)
    abre_facilidadeG[f] = model.addVar(vtype="B", name="abreG[%d]"%f)
    for c in range(m):
      designa[f,c] = model.addVar(vtype="C", name="designa(%d,%d)"%(f+1,c), lb = 0, ub = 1)

  #Assegurar que a capacidade de cada uma das facilidades é respeitada
  for f in range(n):
    model.addCons((scip.quicksum(designa[f,c] * d_j[c] for c in range(m)) <= cap_p*abre_facilidadeP[f] + cap_g*abre_facilidadeG[f]), "Capacidade(%d)"%f)

  #Assegurar que no máximo um tipo de facilidade é construida por local
  for f in range(n):
    model.addCons((abre_facilidadeP[f] + abre_facilidadeG[f]) <= 1, "restricao(%d)"%f)

  #para cada cliente, assegure que sua demanda é atendida
  for c in range(m):
    model.addCons((scip.quicksum(designa[f,c] for f in range(n))) == 1, "Demanda(%d)"%c)

  #Assegura que o projeto está dentro do orçamento
  model.addCons((scip.quicksum(f_g * abre_facilidadeG[f] + f_p * abre_facilidadeP[f] for f in range(n))) <= c_max , "orçamento(%d)"%c)


  model.setObjective((scip.quicksum(designa[f,c]*distancia_percorrida[f,c]*d_j[c] for f in range(n) for c in range(m))), "minimize")

  model.setParam('limits/time', time_limit)

  return model, abre_facilidadeP, abre_facilidadeG, designa

In [75]:
model, abre_facilidadeP, abre_facilidadeG, designa = Cria_modelo_minimizacao(n, m, f_p, f_g, cap_p, cap_g, cap_i, c_max, d_j, distancia_percorrida, 300)

In [76]:
distancia_percorrida[1,1]

56.568542494923804

In [77]:
model.optimize()

presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 10 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 10 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver (2 rounds): 0 aggregations, 0 fixings, 10 bound changes
(round 2, medium)     0 del vars, 0 del conss, 0 add conss, 20 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 10 clqs
(round 3, exhaustive) 0 del vars, 0 del conss, 0 add conss, 20 chg bounds, 0 chg sides, 0 chg coeffs, 11 upgd conss, 0 impls, 10 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int -, cont +), (fixed: bin -, int +, cont -)
   (0.0s) no symmetry present
presolving (4 rounds: 4 fast, 3 medium, 2 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 20 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 10 cliques
presolved problem has 230 variables (20 bin, 0 int, 0 impl, 210 co

In [78]:
# Mostrar quais facilidades foram construídas
for facility in abre_facilidadeP:
    if (abs(abre_facilidadeP[facility].getLPSol()) > 0.5):
      print(f"\n Construiu a facilidade PEQUENA número: {facility + 1}.")


 Construiu a facilidade PEQUENA número: 4.

 Construiu a facilidade PEQUENA número: 5.

 Construiu a facilidade PEQUENA número: 6.

 Construiu a facilidade PEQUENA número: 8.

 Construiu a facilidade PEQUENA número: 10.


In [79]:
# Mostrar quais facilidades foram construídas
for facility in abre_facilidadeG:
  if (abs(abre_facilidadeG[facility].getLPSol()) > 0.5):
    print(f"\n Construiu a facilidade GRANDE número: {facility + 1}.")


 Construiu a facilidade GRANDE número: 1.

 Construiu a facilidade GRANDE número: 2.

 Construiu a facilidade GRANDE número: 3.

 Construiu a facilidade GRANDE número: 7.

 Construiu a facilidade GRANDE número: 9.


In [80]:
# Fornecimento de cada facilidade para os clientes, sendo que cada cliente deve ter sua demanda totalmente atendida
for customer, facility in designa:
    if (abs(designa[customer, facility].getLPSol()) > 1e-6):
        print(f"\n Facilidade {customer} fornece {round(100*designa[customer, facility].getLPSol(), 2)} % da demanda do cliente {facility} .")


 Facilidade 0 fornece 36.59 % da demanda do cliente 0 .

 Facilidade 1 fornece 100.0 % da demanda do cliente 1 .

 Facilidade 1 fornece 100.0 % da demanda do cliente 2 .

 Facilidade 1 fornece 100.0 % da demanda do cliente 3 .

 Facilidade 1 fornece 100.0 % da demanda do cliente 4 .

 Facilidade 1 fornece 100.0 % da demanda do cliente 5 .

 Facilidade 1 fornece 41.67 % da demanda do cliente 12 .

 Facilidade 1 fornece 100.0 % da demanda do cliente 13 .

 Facilidade 2 fornece 14.02 % da demanda do cliente 0 .

 Facilidade 2 fornece 100.0 % da demanda do cliente 6 .

 Facilidade 2 fornece 100.0 % da demanda do cliente 20 .

 Facilidade 3 fornece 100.0 % da demanda do cliente 8 .

 Facilidade 4 fornece 100.0 % da demanda do cliente 9 .

 Facilidade 4 fornece 100.0 % da demanda do cliente 10 .

 Facilidade 5 fornece 100.0 % da demanda do cliente 11 .

 Facilidade 5 fornece 80.56 % da demanda do cliente 17 .

 Facilidade 6 fornece 28.05 % da demanda do cliente 0 .

 Facilidade 6 fornece 58